In [2]:
import os
import google.generativeai as genai
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from config import GEMINI_API_KEY


In [3]:
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

In [18]:
file_path = "data/data.txt"
loader = TextLoader(file_path)
documents = loader.load()

In [19]:
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [20]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)
vectorstore.save_local("my_vector_index")

In [21]:
def query_gemini(question):
    retriever = vectorstore.as_retriever(search_type="similarity", k=3)
    results = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in results])
    prompt = f"""
Answer the question using the following context.

Context:
{context}

Question: {question}
Answer:"""
    response = gemini_model.generate_content(prompt)
    return response.text